# Test notebook

In [1]:
import sys; sys.argv=['']; del sys ### seems to be required to use argparse in ipython. TODO: understand!!
import argparse
import os
import shutil
import time

from resources.transforms import *
from resources.dataset import *
from resources.fp16 import *
from resources.conv_learner import *
from resources.distributed import DistributedDataParallel as DDP

from pathlib import *

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets


### Need adequat __init__.py in folder in order to automatically import objects(?). Without this, __dict__ will contain no
### elements other than standard namespace, hence conditions name.startswith("__") and callable(models.__dict__[name]) will fail
import resources.models as models
import resources.models.cifar10 as cifar10models

In [2]:
model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))

cifar10_names = sorted(name for name in cifar10models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(cifar10models.__dict__[name]))

model_names = cifar10_names + model_names

In [ ]:
print(model_names)
print(len(model_names))

In [62]:
parser = argparse.ArgumentParser(description='PyTorch Cifar10 Training')

parser.add_argument('--arch', '-a', metavar='ARCH', default='resnet56',
                    choices=model_names,
                    help='model architecture: ' +
                    ' | '.join(model_names) +
                    ' (default: resnet56)')

parser.add_argument('--pretrained', dest='pretrained', action='store_true', help='use pre-trained model')


parser.add_argument('--dist-url', default='file://sync.file', type=str,
                    help='url used to set up distributed training')

parser.add_argument('--dist-backend', default='nccl', type=str, help='distributed backend')

parser.add_argument('--world-size', default=1, type=int,
                    help='Number of GPUs to use. Can either be manually set ' +
                    'or automatically set by using \'python -m multiproc\'.')

parser.add_argument('--rank', default=0, type=int,
                    help='Used for multi-process training. Can either be manually set ' +
                    'or automatically set by using \'python -m multiproc\'.')

_StoreAction(option_strings=['--rank'], dest='rank', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help="Used for multi-process training. Can either be manually set or automatically set by using 'python -m multiproc'.", metavar=None)

In [63]:
args_input = [
    '-a', 'preact_resnet50', 
    '--pretrained', 'FLASE'
]

In [64]:
args = parser.parse_args()

In [66]:
print(args.arch)
print(args.rank)
print(args.pretrained)

resnet56
0
False


In [53]:
args.gpu = 0
args.gpu = args.rank % torch.cuda.device_count()
torch.cuda.device_count()

1

In [54]:
args.distributed = args.world_size > 1
args.gpu = 0
if args.distributed:
    args.gpu = args.rank % torch.cuda.device_count()

if args.distributed:
    torch.cuda.set_device(args.gpu)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                            world_size=args.world_size)

# if args.fp16:
#     assert torch.backends.cudnn.enabled, "fp16 mode requires cudnn backend to be enabled."

In [55]:
args.gpu

0

In [67]:
### Source from definition inside clr_resnet.py which was imported at beginning throuh import cifar10models
model = cifar10models.__dict__[args.arch] if args.arch in cifar10_names else models.__dict__[args.arch] 
if args.pretrained:
    print("=> using pre-trained model '{}'".format(args.arch))
    model = model(pretrained=True)
else:
    print("=> creating model '{}'".format(args.arch))
    model = model()

=> creating model 'resnet56'


In [69]:
model_ = cifar10models.__dict__[args.arch] if args.arch in cifar10_names else models.__dict__[args.arch] 
model_()

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
      (conv1): Conv2d(16, 16, kerne